In [2]:
from nltk.util import ngrams
from collections import defaultdict
from collections import OrderedDict
from tkinter import *
import string
import time
import gc
from math import log10
import sqlite3
from sqlite3 import Error
start_time = time.time()
import csv
import pandas as pd
print(csv.__file__)
import glob, os
import requests
import os
import re
from bs4 import BeautifulSoup
import sys, time
from itertools import cycle

global keyword
# membuat koneksi
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return None
# mengecek kalimat yang tidak ada di korpus kata
def cekWrong(conn,inputWrong):
    x = 0
#   memecah inputan menjadi per kata
    temp_l = inputWrong.split()
    i = 0
    j = 0
#   Membersihkan inputan
    for word in temp_l :
        j = 0
        for l in word :
            if l in string.punctuation:
                if l == "'":
                    if j+1<len(word) and word[j+1] == 's':
                        j = j + 1
                        continue
                word = word.replace(l," ")
#                     print(j,word[j])
            j += 1
        temp_l[i] = word.lower()
        i=i+1   
    content = " ".join(temp_l)
    token = content.split()
#   cek perkata ada di korpus kata atau tidak
    for teks in token:
        kata.append(teks)
        teksStemming = fetch(teks)
        cur = conn.cursor()
#       query untuk cek yang tidak memiliki tag
        cur.execute('Select kata from korpus_kata where kata = (?) and tag != "Entri tidak ditemukan" and tag != "tidak ditemukan kata yang dicari" and tag != "entri tidak ditemukan" ',(teksStemming,))
        a = cur.fetchone()
        conn.commit()
        if a==None:
#           kaliamt yang tidak ada di korpus kata disimpan di wrong
            wrong.append(x)
        x= x + 1
# fungsi untuk mengambil 3 kata sebelum kalimat singkatan

def kalimatTeks(i):
    n = wrong[i]
    if n >= 3 :
        temp = kata[n-3] + ' ' + kata[n-2]  + ' ' + kata[n-1]
        input_teks.append(temp)
    elif n == 2 :
        temp = 's' + ' ' + kata[n-2] + ' ' + kata[n-1]
        input_teks.append(temp)
    elif n == 1 :
        temp = 's' + ' ' + 's' + ' ' + kata[n-1]
        input_teks.append(temp)
    elif n == 0 :
        temp = 's' + ' ' + 's' + ' ' + 's'
        input_teks.append(temp)
    print(n)
# fungsi membersihkan kalimat
def removePunctuations(sen):
    temp_l = sen.split()
    i = 0
    j = 0
#   menghilangkan tanda baca dan membuat jadi huruf kecil
    for word in temp_l :
        j = 0
        for l in word :
            if l in string.punctuation:
                if l == "'":
                    if j+1<len(word) and word[j+1] == 's':
                        j = j + 1
                        continue
                word = word.replace(l," ")
            j += 1
        temp_l[i] = word.lower()
        i=i+1   
    content = " ".join(temp_l)
    return content
# memuat korpus kalimat untuk menjadi dataset dan menghitung jumlah dari quadgram, trigram dan bigram 
def loadCorpus(file_path, bi_dict, tri_dict, quad_dict, vocab_dict):
    w1 = ''    #variabel untuk menyimpan 3 kalimat terakhir untuk token set
    w2 = ''    #variabel untuk menyimpan 2 kalimat terakhir untuk token set
    w3 = ''    #variabel untuk menyimpan kalimat terakhir untuk token set
    token = []
    word_len = 0
#   memuat korpus kalimat dan di baca line per line
    with open(file_path,'r', encoding="utf-8") as file:
        print(file)
        for line in file:
            print(line)
#           memecah kalimat menjadi kata
            temp_l = line.split()
            i = 0
            j = 0
#           menghilangkan tanda baca dan mebuat kata menjadi huruf kecil
            for word in temp_l :
                j = 0
                for l in word :
                    if l in string.punctuation:
                        if l == "'":
                            if j+1<len(word) and word[j+1] == 's':
                                j = j + 1
                                continue
                        word = word.replace(l," ")
                    j += 1

                temp_l[i] = word.lower()
                i=i+1   
            content = " ".join(temp_l)
            token = content.split()
#           menghitung jumlah kata
            word_len = word_len + len(token)  
            if not token:
                continue
#           menambahkan kalimat terakhir ke variabel
            if w3!= '':
                token.insert(0,w3)
#           token untuk bigrams
            temp0 = list(ngrams(token,2))
#           menambahkan kalimat dua terakhir ke variabel
            if w2!= '':
                token.insert(0,w2)
#           token untuk trigrams
            temp1 = list(ngrams(token,3))
#           menambahkan kalimat tiga terakhir ke variabel
            if w1!= '':
                token.insert(0,w1)
#           menambahkan kata unik ke vocaulary
            for word in token:
                if word not in vocab_dict:
                    vocab_dict[word] = 1
                else:
                    vocab_dict[word]+= 1
#           token untuk quadgrams
            temp2 = list(ngrams(token,4))
#           menghitung frekuensi dari bigram
            for t in temp0:
                sen = ' '.join(t)
                bi_dict[sen] += 1
#           menghitung frekuensi dari trigram
            for t in temp1:
                sen = ' '.join(t)
                tri_dict[sen] += 1
#           menghitung frekuensi dari quadgrams
            for t in temp2:
                sen = ' '.join(t)
                quad_dict[sen] += 1
#           menghitung panjang token
            n = len(token)
#           menambahkan kalimat ke variabel
            if (n -3) >= 0:
                w1 = token[n -3]
            if (n -2) >= 0:
                w2 = token[n -2]
            if (n -1) >= 0:
                w3 = token[n -1]
    return word_len

# memuat korpus kalimat untuk menjadi dataset character dan menghitung jumlah dari quadgram, trigram dan bigram 
def loadCorpusChar(file_path, bi_dict_char, tri_dict_char, quad_dict_char, vocab_dict_char):
    w1_char = ''    #variabel untuk menyimpan 3 huruf terakhir untuk token set
    w2_char = ''    #variabel untuk menyimpan 2 huruf terakhir untuk token set
    w3_char = ''    #variabel untuk menyimpan huruf terakhir untuk token set
    token = []
    word_len_char = 0
#   memuat korpus kalimat dan di baca line per line
    with open(file_path,'r', encoding="utf-8") as file:
        for line in file:
#           memecah kalimat menjadi karakter
            temp_l_char = list(line)
            i = 0
            j = 0
#           menghilangkan tanda baca dan mebuat huruf kecil
            for char in temp_l_char :
                j = 0
                for l in char :
                    if l in string.punctuation:
                        if l == "'":
                            if j+1<len(char) and char[j+1] == 's':
                                j = j + 1
                                continue
                        char = char.replace(l," ")
                        #print(j,word[j])
                    j += 1

                temp_l_char[i] = char.lower()
                i=i+1   
            content_char = " ".join(temp_l_char)
            token_char = content_char.split()
#           menghitung jumlah karakter
            word_len_char = word_len_char + len(token_char)  
            if not token_char:
                continue
#           menambahkan huruf terakhir ke variabel
            if w3_char!= '':
                token_char.insert(0,w3_char)
#           token untuk bigrams
            temp0_char = list(ngrams(token_char,2))
#           menambahkan 2 huruf terakhir ke variabel
            if w2_char!= '':
                token_char.insert(0,w2_char)
#           token untuk trigrams
            temp1_char = list(ngrams(token_char,3))
#           menambahkan 3 huruf terakhir ke variabel
            if w1_char!= '':
                token_char.insert(0,w1_char)
#           menambahkan huruf unik ke vocaulary
            for char in token_char:
                if char not in vocab_dict_char:
                    vocab_dict_char[char] = 1
                else:
                    vocab_dict_char[char]+= 1
#           token untuk quadgrams
            temp2_char = list(ngrams(token_char,4))
#           menghitung frekuensi dari bigram
            for t in temp0_char:
                sen_char = ''.join(t)
                bi_dict_char[sen_char] += 1
#           menghitung frekuensi dari trigram
            for t in temp1_char:
                sen_char = ''.join(t)
                tri_dict_char[sen_char] += 1
#           menghitung frekuensi dari quadgram
            for t in temp2_char:
                sen_char = ''.join(t)
                quad_dict_char[sen_char] += 1
#           menghitung panjang token
            n_char = len(token_char)
#           menambahkan kalimat ke variabel
            if (n_char -3) >= 0:
                w1_char = token_char[n_char -3]
            if (n_char -2) >= 0:
                w2_char = token_char[n_char -2]
            if (n_char -1) >= 0:
                w3_char = token_char[n_char -1]
    return word_len_char

# membuat dict untuk menyimpan probabilitas kalimat trigrams
def findTrigramProbGT(vocab_dict, bi_dict, tri_dict, tri_prob_dict, nc_dict, k):
    V = len(vocab_dict)
    for tri in tri_dict:
#       membagi kalimat trigrams menjadi perkata
        tri_token = tri.split()
#       membuat kalimat bigram dari kalimat trigrams
        bi_sen = ' '.join(tri_token[:2])
        tri_count = tri_dict[tri]
        bi_count = bi_dict[bi_sen]
#       menggunakan good turing smoothing
        if tri_dict[tri] <= k or (tri not in tri_dict):
            tri_count = findGoodTuringAdjustCount( tri_dict[tri], k, nc_dict)
        if bi_dict[bi_sen] <= k or (bi_sen not in bi_dict):
            bi_count = findGoodTuringAdjustCount( bi_dict[bi_sen], k, nc_dict)
#       menghitung probabilitas trigram dengan membagi kalimat trigram dengan bigram
        prob = tri_count / bi_count       
#       menyimpan probabilitas trigrams ke dict
        if bi_sen not in tri_prob_dict:
            tri_prob_dict[bi_sen] = []
            tri_prob_dict[bi_sen].append([prob,tri_token[-1]])
        else:
            tri_prob_dict[bi_sen].append([prob,tri_token[-1]])
    prob = None
    tri_token = None
    bi_sen = None
# membuat dict untuk menyimpan probabilitas kalimat bigrams
def findBigramProbGT(vocab_dict, bi_dict, bi_prob_dict, nc_dict, k):
    V = len(vocab_dict)
    bigram = []
    bigram_prob = []
    for bi in bi_dict:
#       membagi kaliamat bigrams menjadi perkata
        bi_token = bi.split()
        bigram.append(bi)
#       membuat kalimat unigrams dari kalimat bigrams
        unigram = bi_token[0]
        bi_count = bi_dict[bi]
        uni_count = vocab_dict[unigram]    
#       menggunakan goot turing smoothing
        if bi_dict[bi] <= k or (bi not in bi_dict):
            bi_count = findGoodTuringAdjustCount( bi_dict[bi], k, nc_dict)
        if vocab_dict[unigram] <= k or (unigram not in vocab_dict):
            uni_count = findGoodTuringAdjustCount( vocab_dict[unigram], k, nc_dict)
#       menghitung probabilitas bigram dengan membagi kalimat bigram dengan unigram
        prob = bi_count / uni_count
        bigram_prob.append(prob)
#       menyimpan probabilitas bigrams ke dict
        if unigram not in bi_prob_dict:
            bi_prob_dict[unigram] = []
            bi_prob_dict[unigram].append([prob,bi_token[-1]])
        else:
            bi_prob_dict[unigram].append([prob,bi_token[-1]])
    prob = None
    bi_token = None
    unigram = None

# membuat dict untuk menyimpan probabilitas karakter trigrams
def findCharTrigramProbGT(vocab_dict_char, bi_dict_char, tri_dict_char, tri_prob_dict_char, nc_dict_char, k):
    V = len(vocab_dict_char)
    trigram = []
    trigram_prob = []    
    for tri_char in tri_dict_char:
#       membagi kaliamat trigrams menjadi perhuruf
        tri_token_char = tri_char.split()
        char = ' '.join(tri_token_char[0])
        tri_tokenc = char.split()
#       membuat huruf bigram dari huruf trigrams
        bi_sen_char = ''.join(tri_tokenc[:2])
        tri_count_char = tri_dict_char[tri_char]
        bi_count_char = bi_dict_char[bi_sen_char]    
#       menggunakan goot turing smoothing
        if tri_dict_char[tri_char] <= k or (tri_char not in tri_dict_char):
            tri_count_char = findGoodTuringAdjustCount( tri_dict_char[tri_char], k, nc_dict_char)
        if bi_dict_char[bi_sen_char] <= k or (bi_sen_char not in bi_dict_char):
            bi_count_char = findGoodTuringAdjustCount( bi_dict_char[bi_sen_char], k, nc_dict_char)
#       menghitung probabilitas trigram dengan membagi huruf trigram dengan bigram
        prob_char = tri_count_char / bi_count_char
        trigram_prob.append(prob_char)
        tri_prob_dict_char[tri_char] = prob_char        
#       menyimpan probabilitas trigrams ke dict
        if bi_sen_char not in tri_prob_dict_char:
            tri_prob_dict_char[bi_sen_char] = []
            tri_prob_dict_char[bi_sen_char].append([prob_char,tri_token_char[-1]])
        else:
            tri_prob_dict_char[bi_sen_char].append([prob_char,tri_token_char[-1]])
    prob_char = None
    tri_token_char = None
    bi_sen_char = None
# membuat dict untuk menyimpan probabilitas karakter bigrams
def findCharBigramProbGT(vocab_dict_char, bi_dict_char, bi_prob_dict_char, nc_dict_char, k):
    #vocabulary size
    V = len(vocab_dict_char)
    bigram = []
    bigram_prob = []
    for bi_char in bi_dict_char:
#       membagi kaliamat bigrams menjadi perhuruf
        bi_token_char = bi_char.split()
        bigram.append(bi_char)
#       membuat huruf unigram dari huruf bigrams
        unigram_char = bi_token_char[0][0]
        test.append(unigram_char)
        bi_count_char = bi_dict_char[bi_char]
        uni_count_char = vocab_dict_char[unigram_char]
#       menggunakan goot turing smoothing
        if bi_dict_char[bi_char] <= k or (bi_char not in bi_dict_char):
            bi_count_char = findGoodTuringAdjustCount( bi_dict_char[bi_char], k, nc_dict_char)
        if vocab_dict_char[unigram_char] <= k or (unigram_char not in vocab_dict_char):
            uni_count_char = findGoodTuringAdjustCount( vocab_dict_char[unigram_char], k, nc_dict_char)
#       menghitung probabilitas bigram dengan membagi huruf bigram dengan unigram
        prob_char = bi_count_char / uni_count_char
        bigram_prob.append(prob_char)
        bi_prob_dict_char[bi_char] = prob_char
#       menyimpan probabilitas bigrams ke dict
        if unigram_char not in bi_prob_dict_char:
            bi_prob_dict_char[unigram_char] = []
            bi_prob_dict_char[unigram_char].append([prob_char,bi_token_char[-1]])
        else:
            bi_prob_dict_char[unigram_char].append([prob_char,bi_token_char[-1]])
    prob_char = None
    bi_token_char = None
    unigram_char = None

# untuk mengurutkan probabilitas dari yang terbesar
def sortProbWordDict(bi_prob_dict, tri_prob_dict, quad_prob_dict):
    for key in bi_prob_dict:
        if len(bi_prob_dict[key])>1:
            bi_prob_dict[key] = sorted(bi_prob_dict[key],reverse = True)
    for key in tri_prob_dict:
        if len(tri_prob_dict[key])>1:
            tri_prob_dict[key] = sorted(tri_prob_dict[key],reverse = True)
    for key in quad_prob_dict:
        if len(quad_prob_dict[key])>1:
            quad_prob_dict[key] = sorted(quad_prob_dict[key],reverse = True)[:2]

# untuk mengambil inputan
def takeInput():
    cond = False
    while(cond == False):
        sen = input('Enter the string\n')
        temp = sen.split()
        if len(temp) < 0:
            print("Please enter atleast 1 words !")
        else:
            cond = True
            temp = temp[:]
    sen = " ".join(temp)
    return sen
# mencari adjusted count c* di good turing smoothing
def findGoodTuringAdjustCount(c, k, nc_dict):
    adjust_count = ( ( (( c + 1)*( nc_dict[c + 1] / nc_dict[c])) - ( c * (k+1) * nc_dict[k+1] / nc_dict[1]) ) /
                     ( 1 - (( k + 1)*nc_dict[k + 1] / nc_dict[1]) )
                   )
    return adjust_count

# mencari prediksi kalimat menggunakan backoff
def doPredictionBackoffGT(input_sen, bi_dict, tri_dict, quad_dict, bi_prob_dict, tri_prob_dict, quad_prob_dict):
    token = input_sen.split()
    if input_sen in quad_prob_dict and quad_prob_dict[ input_sen ][0][0]>0:
        print("quad")
        pred = quad_prob_dict[input_sen][0]
    elif ' '.join(token[1:]) in tri_prob_dict and tri_prob_dict[' '.join(token[1:])][0][0]>0:
        print("tri")
        pred = tri_prob_dict[ ' '.join(token[1:]) ][0]
    elif ' '.join(token[2:]) in bi_prob_dict and bi_prob_dict[ ' '.join(token[2:]) ][0][0]>0:
        print("bi")
        pred = bi_prob_dict[' '.join(token[2:])][0]
    else:
        print("kosong")
        pred = []
    return pred

# fungsi mencari kemungkiann kata dari kata singkatan
def contained(pred,kata):
    jumlahs = len(pred)
    jumlaht = len(kata)
    jumlah = 0
    value = 0
    i=0
    j=0
    n = ""
    veri = 0
#   untuk kata yang berulang seperti yaaaaaannnnggg menjadi yang
    if len(kata) > 2: 
        for k in range(0,len(kata)):
            if len(kata) > 0:
                if k < len(kata)-1:
                    if kata[k] != kata[k+1]:
                        if value == 1:
                            n = n + kata[k] + kata[k]
                            veri = 1
                            value = 0
                        else: 
                            n = n + kata[k]
                            veri = 1
                            value = 0
                    elif kata[k] == kata[k+1]:
                        value+=1
                        if k == len(kata)-2:
                            if value == 1:
                                n = n + kata[k]
        if veri != 0:
            n = n + kata[len(kata)-1]
            kata = n
            jumlaht = len(kata)
#   menghitung berapa kalimat singkatan yang mengandung kalimt prediksi
    while i < jumlaht:
        while j < jumlahs:
            if kata[i]==pred[j]:
                jumlah+=1
                j+=1
                break
            j+=1
        i+=1    
#   mengembalikan nilai true apa bila semua mengandung kata tersebut
    if jumlah >= jumlaht:
        return True
    else :
        return False
# membuat gui
# membuat form
def makeform(root, field):
    print(field)
    row = Frame(root)
    lab = Label(row, width=22, text=field+": ", anchor='w')
    ent = Text(row, height=5)
    row.pack(side=TOP, 
             fill=X, 
             padx=5, 
             pady=5)
    lab.pack(side=LEFT)
    ent.pack(side=RIGHT, 
             expand=YES, 
             fill=X)
    entries[field] = ent
    return entries
# membuat tabel
def maketable(root,kataslang,prediksi):
    b = Frame(root)
    b.pack(side=TOP, 
             fill=X, 
             padx=5, 
             pady=5)
    n = len(kataslang)
    count = 0
    height = 8
    width = 2
    for i in range(n): #Rows
        for j in range(width): #Columns
            tab = Entry(b, text="")
            if j == 0:
                tab.insert(0, kataslang[count])
            else:
                tab.insert(0, prediksi[count])
            tab.grid(row=i, column=j)
        count+=1
    return tab
            
def clicked(entries,senInput):
    entries['Output'].insert('0.0', senInput[:] )
token = []
kata = []
index = []
wrong = []
input_teks = []
test = []
true = []
prediksi = []
kataslang = []
dataCsv = []
senInput = []
entries = {}
from tkinter import *
colomn_wrong = []
colomn_pred = []
colomn_prob = []
if __name__ == '__main__':
    vocab_dict = defaultdict(int)          #untuk membuat kalimat yang unik menjadi ada frekuensi   
    bi_dict = defaultdict(int)             #untuk menyimpan 2 kalimat 
    tri_dict = defaultdict(int)            #untuk menyimpan 3 kalimat 
    quad_dict = defaultdict(int)           #untuk menyimpan 4 kalimat 
    vocab_dict_char = defaultdict(int)          
    bi_dict_char = defaultdict(int)             
    tri_dict_char = defaultdict(int)           
    quad_dict_char = defaultdict(int)           
    quad_prob_dict = OrderedDict()              
    tri_prob_dict = OrderedDict()
    bi_prob_dict = OrderedDict()
    quad_prob_dict_char = OrderedDict()              
    tri_prob_dict_char = OrderedDict()
    bi_prob_dict_char = OrderedDict() 
    list_pred = []
#   koneksi ke db
    database = "C:\\db\\korpus.db"
    conn = create_connection(database)
#   membuka korpus kecil
    train_file = 'Kecilkorpus.txt'
#   menjalankan fungsi loadcorpus dan loadcorpuschar
    token_len = loadCorpus(train_file, bi_dict, tri_dict, quad_dict, vocab_dict)
    token_len_char = loadCorpusChar(train_file, bi_dict_char, tri_dict_char, quad_dict_char, vocab_dict_char)
    k = 5
    V = len(vocab_dict)
    tri_nc_dict = findFrequencyOfFrequencyCount(tri_dict, k, 3, V, len(tri_dict))
    bi_nc_dict = findFrequencyOfFrequencyCount(bi_dict, k, 2, V, len(bi_dict))
    uni_nc_dict = findFrequencyOfFrequencyCount(bi_dict, k, 1, V, len(vocab_dict))
    tri_nc_dict_char = findFrequencyOfFrequencyCount(tri_dict_char, k, 3, V, len(tri_dict_char))
    bi_nc_dict_char = findFrequencyOfFrequencyCount(bi_dict_char, k, 2, V, len(bi_dict_char))
    uni_nc_dict_char = findFrequencyOfFrequencyCount(bi_dict_char, k, 1, V, len(vocab_dict_char))
#   membuat dic probabilitas bigram, trigram, charbigram, chartrigram
    findBigramProbGT(vocab_dict, bi_dict, bi_prob_dict, bi_nc_dict, k)
    findTrigramProbGT(vocab_dict, bi_dict, tri_dict, tri_prob_dict, tri_nc_dict, k) 
    findCharBigramProbGT(vocab_dict_char, bi_dict_char, bi_prob_dict_char, bi_nc_dict_char, k)
    findCharTrigramProbGT(vocab_dict_char, bi_dict_char, tri_dict_char, tri_prob_dict_char, tri_nc_dict_char, k)
    sortProbWordDict(bi_prob_dict, tri_prob_dict, quad_prob_dict)
    entries = {}
    root = Tk()
    ents = makeform(root, 'Input')
    b1 = Button(root, text='Normalisasi', command=(lambda e=ents: main(e,root)))
    b1.pack(side=TOP, fill=X, padx=5, pady=5)
    ents = makeform(root, 'Output') 
    root.mainloop()



C:\Users\Kijeng\AppData\Local\Programs\Python\Python38\lib\csv.py


FileNotFoundError: [Errno 2] No such file or directory: 'banjir.txt'